In [ ]:
!pip install transformers
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import os
from sklearn.model_selection import train_test_split


logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere


max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = [0,1]

In [ ]:
def prepare_dataset():

    train = pd.read_csv('https://raw.githubusercontent.com/GIL-UNAM/PARMEX_2022/main/parmex_train.csv')
    valid = pd.read_csv('https://raw.githubusercontent.com/GIL-UNAM/PARMEX_2022/main/parmex_validation.csv')
    print("Train Dataframe:")
    train.head(3)
    print(f'Train dataframe contains {train.shape[0]} samples.')
    print('Number of features in train data : ', train.shape[1])
    print('Train Features : ', train.columns.values)

    train_df,test_df=train_test_split(train,test_size=0.25,random_state=10)

    valid_df=test_df

    train_df = (
        train_df[train_df.Label != "-"]
        .sample(frac=1.0, random_state=42)
        .reset_index(drop=True)
    )
    valid_df = (
        valid_df[valid_df.Label != "-"]
        .sample(frac=1.0, random_state=42)
        .reset_index(drop=True)
    )

    y_train = tf.keras.utils.to_categorical(train_df.Label, num_classes=2)


    y_val = tf.keras.utils.to_categorical(valid_df.Label, num_classes=2)


    y_test = tf.keras.utils.to_categorical(test_df.Label, num_classes=2)

    return train_df, valid_df, valid_df, y_train, y_val, y_test


In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
        model_path=''
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.model_path=model_path
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            model_path, do_lower_case=True##################################################################################################
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [ ]:
# Create the model under a distribution strategy scope.
def create_model(model_path):
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        # Encoded token ids from BERT tokenizer.
        input_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="input_ids"
        )
        # Attention masks indicates to the model which tokens should be attended to.
        attention_masks = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="attention_masks"
        )
        # Token type ids are binary masks identifying different sequences in the model.
        token_type_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="token_type_ids"
        )
        # Loading pretrained BERT model.
        bert_model = transformers.TFBertModel.from_pretrained(model_path)########################################################################
        # Freeze the BERT model to reuse the pretrained features without modifying them.
        bert_model.trainable = False

        bert_output = bert_model(
            input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
        )
        sequence_output = bert_output.last_hidden_state
        pooled_output = bert_output.pooler_output
        # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
        bi_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True)
        )(sequence_output)
        # Applying hybrid pooling approach to bi_lstm sequence output.
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)
        output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
        model = tf.keras.models.Model(
            inputs=[input_ids, attention_masks, token_type_ids], outputs=output
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["acc"],
        )


    print(f"Strategy: {strategy}")
    model.summary()

    return model, bert_model


In [ ]:
def create_bert_semantic_data(data_set, text1, text2, y, model_path):
    semantic_data = BertSemanticDataGenerator(
    data_set[[text1, text2]].values.astype("str"),
    y,
    batch_size=batch_size,
    shuffle=True,
    model_path=model_path
    )


    return semantic_data





In [ ]:
def fit_module(model, train, valid, epochs):

    history = model.fit(
    train,
    validation_data=valid,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    )

    return history, model


In [ ]:
def  unfreeze_recompile_model(bert_model, model):
    bert_model.trainable = True

    model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    )
    model.summary()

    return model.summary(), model


In [ ]:

def fit_model(model, train, valid, epochs):
    history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    )

    return history, model



In [ ]:
def validate_model(model, test_data, verbose=1):

    return model.evaluate(test_data,verbose)





In [ ]:
def check_similarity(model_path ,sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,model_path=model_path
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

In [ ]:
model_list  =['dccuchile/bert-base-spanish-wwm-cased','hiiamsid/sentence_similarity_spanish_es','sentence-transformers/bert-base-nli-mean-tokens']


print ('here now')




tokenizer=''
bert_model=''
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

valid = pd.read_csv('https://raw.githubusercontent.com/GIL-UNAM/PARMEX_2022/main/parmex_validation.csv')

for modx in model_list:
    print("now doing this model")
    print(modx)
    model_path = '/tokenizer2/'+modx
    bert_model = transformers.TFBertModel.from_pretrained(modx,from_pt=True)
    bert_model.save_pretrained(model_path)
    tokenizer = transformers.BertTokenizer.from_pretrained(modx)
    tokenizer.save_pretrained(model_path)

    model, bert_model =create_model(model_path)
    train_df, valid_df, test_df, y_train, y_val, y_test = prepare_dataset()

    train_data = create_bert_semantic_data(train_df, "Text1", "Text2", y_train, model_path=model_path)

    valid_data = create_bert_semantic_data(valid_df, "Text1", "Text2", y_train,model_path=model_path)

    test_data = create_bert_semantic_data(test_df, "Text1", "Text2", y_train, model_path=model_path)
    

    history, model = fit_model(model, train_data, valid_data, epochs)

    history,model = unfreeze_recompile_model(bert_model, model)

    history, model = fit_model(model,train_data, valid_data,epochs)


    predictionst = np.argmax(model.predict(test_data), axis=-1)
    
    
    
    ppp=[]
    for i in range(len(train_df)):
        
        ppp.append(check_similarity(model_path,train_df['Text1'][i], train_df['Text2'][i])[0])
    
    train_df['predictions'] = ppp
    train_df.to_csv(modx.split("/")[1]+'misclassifyTrain.csv', header=True, index=False, columns=list(train_df.axes[1]))
    
    ppp=[]
    for i in range(len(test_df)):
        ppp.append(check_similarity(model_path,test_df['Text1'][i], test_df['Text2'][i])[0])
    test_df['predictions'] = ppp
    test_df.to_csv(modx.split("/")[1]+'misclassifyTest.csv', header=True, index=False, columns=list(test_df.axes[1]))
    
    ppp=[]
    for i in range(len(valid)):
        ppp.append(check_similarity(model_path,valid['Text1'][i], valid['Text2'][i])[0])
    valid['predictions'] = ppp
    valid.to_csv(modx.split("/")[1]+'misclassifyTest.csv', header=True, index=False, columns=list(valid.axes[1]))
    
    

    '''h_df = pd.DataFrame(predictionst)
    hist_csv_file = modx+'history.csv'
    with open(hist_csv_file, mode='w') as f:
        h_df.to_csv(f)
    
    predictionsv  = np.argmax(model.predict(test_data), axis=-1)

    h_df = pd.DataFrame(predictionsv)
    hist_csv_file = modx+'historyvalid.csv'
    with open(hist_csv_file, mode='w') as f:
        h_df.to_csv(f)
    '''
    




    






